# Download and Prepare MusicMIDI for COMMU Processing

In [1]:
!pip install boto3
!pip install python-dotenv

In [5]:
import os
import boto3
import joblib
from ast import literal_eval


from tqdm.notebook import tqdm

import dotenv
dotenv.load_dotenv()

Python-dotenv could not parse statement starting at line 3


True

In [6]:
# declare some parameters
## access secret keys
access_key_id = os.environ.get("S3_KEY_ID")
secret_access_key = os.environ.get("S3_ACCESS_KEY")


# s3 bucket name
bucket_name = "orchestrate-bucket"
bucket_name = 'commu-model-checkpoint'

# output dir
output_dir = "../dataset/processed_musicMIDI/"

In [7]:

s3 = boto3.client("s3",
                  region_name="us-west-2",
                  aws_access_key_id=access_key_id,
                  aws_secret_access_key=secret_access_key)

In [8]:
# push data to s3
def push_to_s3(local_file, bucket_name, s3_file):
    s3.upload_file(local_file, bucket_name, s3_file)

push_to_s3("../data/checkpoint_best_metaMIDI.pt", bucket_name, "checkpoint_best_metaMIDI.pt")

In [11]:
# download data from s3
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket_name, Prefix='metamidi_midi_files')

for page in tqdm(pages):
    files = page['Contents']
    # download all midi files into local
    for file in files:
        file_key = file["Key"]

        # pass things that's not true midi files
        if not file_key.endswith(".mid"):
            continue

        # get a file and download it to local
        response = s3.get_object(Bucket=bucket_name, Key=file_key)
        
        # Read the file from S3
        try:
            response = s3.download_file(bucket_name, file_key, f'{output_dir}/{file_key}')
            # print('File downloaded successfully.')
        except Exception as e:
            print('Error reading file from S3:', e)

0it [00:00, ?it/s]

In [ ]:
# pull specific file from s3
def pull_from_s3(bucket_name, s3_file, local_file):
    s3.download_file(bucket_name, s3_file, local_file)